In [3]:
import os
import re
import csv
import h5py
import warnings
import tqdm
import numpy as np
import pandas as pd

from VSS_DB_utils import VSS_File

In [4]:
# para carregar os dados e iniciar o dataset

datasetPath = "D:/Rafael/Dados/dataset.hdf5"

dataset = VSS_File(datasetPath)

In [6]:
# o dataset possui no topo os 5 compressores que foram testados
dataset.units
#  além disso o dataset é totalmente iterável em loops

[Vibration-based database for unit </1> (1668 tests),
 Vibration-based database for unit </2> (1138 tests),
 Vibration-based database for unit </3> (1186 tests),
 Vibration-based database for unit </4> (1184 tests),
 Vibration-based database for unit </5> (1187 tests)]

In [9]:
# definindo os parâmetros de filtragem abaixo

filterAttributes = {
    "angularSpeed": [3600],
    "condensingTemperature": [30, 40]
}

# os parâmetros são "angular speed", "condensingTemperature" e "evaporatingTemperature"

# a filtragem já retorna uma lista iterável
dataset.DataframeAsList(filterAttributes)

[Vibration soft sensing test database </1/A1-3600-1-[-10,00]-[34,00]>,
 Vibration soft sensing test database </1/A1-3600-1-[-10,00]-[36,00]>,
 Vibration soft sensing test database </1/A1-3600-1-[-10,00]-[38,00]>,
 Vibration soft sensing test database </1/A1-3600-1-[-10,00]-[40,00]>,
 Vibration soft sensing test database </1/A1-3600-1-[-12,00]-[34,00]>,
 Vibration soft sensing test database </1/A1-3600-1-[-12,00]-[36,00]>,
 Vibration soft sensing test database </1/A1-3600-1-[-12,00]-[38,00]>,
 Vibration soft sensing test database </1/A1-3600-1-[-12,00]-[40,00]>,
 Vibration soft sensing test database </1/A1-3600-1-[-14,00]-[34,00]>,
 Vibration soft sensing test database </1/A1-3600-1-[-14,00]-[36,00]>,
 Vibration soft sensing test database </1/A1-3600-1-[-14,00]-[38,00]>,
 Vibration soft sensing test database </1/A1-3600-1-[-14,00]-[40,00]>,
 Vibration soft sensing test database </1/A1-3600-1-[-16,00]-[34,00]>,
 Vibration soft sensing test database </1/A1-3600-1-[-16,00]-[36,00]>,
 Vibra

In [13]:
# sempre dá pra acessar o elemento por meio de indexação
dataset.units[0].tests[0]

Vibration soft sensing test database </1/A1-2100-1-[-10,00]-[34,00]>

In [17]:
# fiz também umas funções pra retornar os dataframes dos ensaios

# a seguinte função retorna como numpy array
dataset.units[0].tests[0].returnNumericalDatabase()

# a seguinte já retorna como dataframe com os headers
dataset.units[0].tests[0].returnNumericalDataframe()

,rpm,t_evap_ref,t_cond_ref,t_evap,t_cond,t_suc,t_comp,t_dis,p_suc,p_int,p_dis
0,2100.0,-10.0,34.0,-10.0641,33.9262,29.7145,51.5706,57.9010,1.07799,3.29912,4.51336
1,2100.0,-10.0,34.0,-10.0681,33.8721,29.7868,51.7261,58.1109,1.07783,3.29878,4.50667
2,2100.0,-10.0,34.0,-10.0693,33.8408,30.1132,52.0800,58.1518,1.07778,3.29864,4.50280
3,2100.0,-10.0,34.0,-10.0719,33.7627,30.0912,51.7100,57.7116,1.07767,3.29836,4.49317
4,2100.0,-10.0,34.0,-10.0724,33.7171,30.0886,51.6939,57.8140,1.07765,3.29815,4.48755
5,2100.0,-10.0,34.0,-10.0757,33.6836,29.8887,51.3025,57.4557,1.07751,3.29751,4.48343
6,2100.0,-10.0,34.0,-10.0780,33.6618,29.9429,51.4205,57.5991,1.07742,3.29710,4.48074
7,2100.0,-10.0,34.0,-10.0827,33.6413,29.8874,51.2917,57.2152,1.07722,3.29623,4.47822
8,2100.0,-10.0,34.0,-10.0857,33.6388,30.0267,51.2542,56.7288,1.07710,3.29571,4.47792
9,2100.0,-10.0,34.0,-10.0886,33.6231,29.9287,51.0129,56.3499,1.07697,3.29531,4.47599


In [18]:
# o mesmo pode ser feito pra vibração

# a seguinte função retorna como numpy array
dataset.units[0].tests[0].returnVibrationDatabase()

# a seguinte já retorna como dataframe com os headers
dataset.units[0].tests[0].returnVibrationDataframe()

,x,y,z
0,0.000020,0.000020,0.000020
1,-0.000007,-0.000009,0.000005
2,-0.000015,-0.000021,0.000008
3,-0.000022,-0.000011,0.000001
4,-0.000023,0.000004,-0.000003
...,...,...,...
511996,0.000037,-0.000242,-0.000571
511997,0.000032,-0.000244,-0.000591
511998,0.000035,-0.000235,-0.000605
511999,0.000051,-0.000234,-0.000608


In [68]:
filterAttributes = {
    "angularSpeed": [3600],
    "condensingTemperature": [30, 40]
}

# vib_data = dataset.DataframeAsList(filterAttributes)[0].splitVibrationWaveform(10,"x")

# vib_data[9]

# selectedUnits=["/1","/2","/3","/4","/5"]

# aaa = (dataset.units[0].filterTestsByAttributeDict(filterAttributes)[0].returnVibrationDatabase())

# pd.Series(dataset.units[0].filterTestsByAttributeDict(filterAttributes)[0]._h5ref["vibrationMeasurements"][:,1])

# np.mean(np.array(dataset.DataframeAsList(filterAttributes)[0]._h5ref["numericalMeasurements"]), axis=0)

# dataset.returnDataframe(filterAttributes)

# list(dataset.units[0].tests[0]._h5ref.attrs)


array([9.860402e-05, 9.648752e-05, 7.579776e-05, ..., 1.218000e-03,
       1.217000e-03, 1.188000e-03])

In [4]:
filterAttributes = {
    "angularSpeed": [2100, 3600],
    "condensingTemperature": [32, 36]
}

dataList = dataset.DataframeAsList(filterAttributes)

len(dataList)

# for test in dataList:
#     print(test.returnNumericalDatabase())


# numData = np.array([np.mean(test.returnNumericalDatabase(),axis=0) for test in dataList])
# # df = pd.DataFrame(numData, columns=dataList[0].returnNumericalHeaders())

# vibData = [pd.Series(test._h5ref["vibrationMeasurements"][:,1], dtype='float16') \
#             for test in tqdm.tqdm(dataList)]

# vib = []
# for test in tqdm.tqdm(dataList):
#     vib.append(test.returnVibrationDatabase()[:,1])
    


# doubled = np.array(doubled)


# (np.mean(dataList[0].returnNumericalDatabase(),axis=0))

# dataList[0].returnVibrationDatabase()[:,1]

# np.mean(np.array(dataList[0]._h5ref["numericalMeasurements"]), axis=0)

# df = pd.DataFrame((np.mean(np.array(dataList[0]._h5ref["numericalMeasurements"]), axis=0)), columns = dataList[0].returnNumericalHeaders())


# pd.DataFrame.from_records()

# [df.join(np.mean(np.array(test._h5ref["numericalMeasurements"]), axis=0)) for test in dataList]




704

In [7]:
df = pd.DataFrame({'vib_x': [pd.Series(test._h5ref["vibrationMeasurements"][:,1], dtype='float32') for test in tqdm.tqdm(dataList)]})

100%|██████████| 704/704 [01:07<00:00, 10.49it/s]


In [8]:
df.memory_usage(index=True, deep=True).sum()/(1024**3)


1.3428758382797241

In [52]:
del df